**Module**

In [1]:
import pandas as pd
import numpy as np
from configparser import ConfigParser
from mysql.connector import MySQLConnection,Error

**Config reading and parsing**

In [2]:
# reading database credentials from config.ini file

def read(filename='config.ini',section='mysql'):
    parser=ConfigParser()
    parser.read(filename)
    
    db={}
    
    if parser.has_section(section):
        items=parser.items(section)
        for item in items:
            db[item[0]]=item[1]
    else:
        raise Exception(f'{section} not found in file {filename}')
    return db 


print(read(filename="config.ini",section="mysql"))

{'host': '127.0.0.1', 'port': '99966', 'database': 'books', 'user': 'root', 'password': 'Milla123!'}


**Connecting with MySQL/MariaDB database server and getting the connection and cursor object**

In [3]:
def connect(creds):
    con=None
    try:
        print('Connecting to MySQL database...')
        con=MySQLConnection(**creds)
        
        
        if con.is_connected():
            print('Connection established')
            cus = con.cursor(buffered=True)
        else:
            print('Connection failed')
            
    except Error as e:
        print(e)
    finally:
        return con,cus
    
cn,cs=connect(creds=read(filename="config.ini",section="mysql"))

Connecting to MySQL database...
Connection established


### Getting top 20 most famous books of all time

- Sort by book_ratings_count and then sort by book_average_rating

#### SQL + Python

In [4]:
sql_query = """
SELECT gr_book_id, book_title, book_ratings_count, book_average_rating
FROM book
ORDER BY book_ratings_count DESC
LIMIT 100;
"""

In [5]:
cs.execute(sql_query)

In [6]:
top100_by_ratings_count = cs.fetchall()

In [7]:
for i in top100_by_ratings_count[:20]:
    print(i)

(2767052, 'The Hunger Games (The Hunger Games, #1)', 4899965, 4.34)
(3, "Harry Potter and the Sorcerer's Stone (Harry Potter, #1)", 4765497, 4.45)
(41865, 'Twilight (Twilight, #1)', 3941381, 3.57)
(2657, 'To Kill a Mockingbird', 3255518, 4.26)
(4671, 'The Great Gatsby', 2758812, 3.89)
(11870085, 'The Fault in Our Stars', 2429317, 4.26)
(5907, 'The Hobbit', 2099680, 4.25)
(5107, 'The Catcher in the Rye', 2086945, 3.79)
(1885, 'Pride and Prejudice', 2078406, 4.25)
(960, 'Angels & Demons  (Robert Langdon, #1)', 2046499, 3.86)
(48855, 'The Diary of a Young Girl', 2028299, 4.1)
(5470, '1984', 2023937, 4.14)
(13335037, 'Divergent (Divergent, #1)', 1962813, 4.23)
(7613, 'Animal Farm', 1928931, 3.88)
(5, 'Harry Potter and the Prisoner of Azkaban (Harry Potter, #3)', 1876252, 4.53)
(2429135, 'The Girl with the Dragon Tattoo (Millennium, #1)', 1858152, 4.11)
(6148028, 'Catching Fire (The Hunger Games, #2)', 1854746, 4.3)
(77203, 'The Kite Runner', 1848782, 4.26)
(15881, 'Harry Potter and the Cha

In [8]:
avg_ratings_top100 = list()

for i in top100_by_ratings_count:
    avg_ratings_top100.append(i[3])

print(avg_ratings_top100)

[4.34, 4.45, 3.57, 4.26, 3.89, 4.26, 4.25, 3.79, 4.25, 3.86, 4.1, 4.14, 4.23, 3.88, 4.53, 4.11, 4.3, 4.26, 4.38, 4.34, 4.53, 4.62, 4.47, 4.03, 4.54, 3.74, 3.65, 3.78, 4.19, 4.45, 3.84, 3.79, 4.23, 4.45, 3.66, 3.82, 4.12, 4.08, 4.04, 4.1, 3.51, 4.36, 4.12, 3.52, 3.69, 3.86, 4.15, 4.07, 3.7, 3.88, 4.07, 3.97, 4.29, 3.88, 3.94, 3.8, 4.2, 4.02, 3.83, 4.21, 4.28, 3.85, 4.06, 4.06, 4.07, 4.34, 4.3, 3.76, 4.18, 4.08, 3.93, 4.28, 3.84, 3.75, 3.95, 4.03, 4.06, 4.37, 4.3, 3.73, 3.7, 4.06, 3.95, 4.12, 4.07, 3.82, 4.24, 3.98, 4.22, 4.04, 4.25, 4.27, 4.06, 4.34, 4.03, 4.1, 3.97, 4.03, 4.22, 3.98]


In [9]:
sorted_by_ratings_indices = np.argsort(np.array(avg_ratings_top100))[::-1]
sorted_by_ratings_indices

array([21, 24, 20, 14, 22,  1, 29, 33, 18, 77, 41, 19, 65, 93,  0, 78, 66,
       16, 52, 71, 60, 91,  5, 17,  3,  8, 90,  6, 86, 32, 12, 98, 88, 59,
       56, 28, 68, 46, 11, 42, 36, 83, 15, 39, 10, 95, 37, 69, 47, 64, 50,
       84, 76, 62, 92, 63, 81, 89, 38, 94, 75, 23, 97, 57, 99, 87, 96, 51,
       82, 74, 54, 70,  4, 49, 53, 13,  9, 45, 61, 72, 30, 58, 35, 85, 55,
        7, 31, 27, 67, 73, 25, 79, 48, 80, 44, 34, 26,  2, 43, 40],
      dtype=int64)

In [10]:
for i in sorted_by_ratings_indices[:20]:
    print(top100_by_ratings_count[i])

(136251, 'Harry Potter and the Deathly Hallows (Harry Potter, #7)', 1784684, 4.62)
(1, 'Harry Potter and the Half-Blood Prince (Harry Potter, #6)', 1713866, 4.54)
(6, 'Harry Potter and the Goblet of Fire (Harry Potter, #4)', 1792561, 4.53)
(5, 'Harry Potter and the Prisoner of Azkaban (Harry Potter, #3)', 1876252, 4.53)
(2, 'Harry Potter and the Order of the Phoenix (Harry Potter, #5)', 1766895, 4.47)
(3, "Harry Potter and the Sorcerer's Stone (Harry Potter, #1)", 4765497, 4.45)
(4667024, 'The Help', 1565302, 4.45)
(13496, 'A Game of Thrones (A Song of Ice and Fire, #1)', 1359501, 4.45)
(15881, 'Harry Potter and the Chamber of Secrets (Harry Potter, #2)', 1821802, 4.38)
(370493, 'The Giving Tree', 720582, 4.37)
(19063, 'The Book Thief', 1193697, 4.36)
(34, 'The Fellowship of the Ring (The Lord of the Rings, #1)', 1813229, 4.34)
(128029, 'A Thousand Splendid Suns', 835172, 4.34)
(3777732, 'City of Glass (The Mortal Instruments, #3)', 606729, 4.34)
(2767052, 'The Hunger Games (The Hunger

#### Only SQL with Stored Procedure

**We can do the same with the following query**

```
CREATE TEMPORARY TABLE sort_by_rating_count AS
SELECT gr_book_id, book_title, book_ratings_count, book_average_rating
FROM book
ORDER BY book_ratings_count DESC
LIMIT 100;

SELECT * FROM sort_by_rating_count
ORDER BY book_average_rating DESC
LIMIT 20;

DROP TEMPORARY TABLE IF EXISTS sort_by_rating_count;
```

**Custom function to call the storted procedure to fetch top 20 books**

In [11]:
def get_top_recommendation_overall(cs,
                   proc_name = "sp_get_top_recommendation_overall"
                   ):

        procs_args_complete = []
        
        proc_args_initial=[0]
        procs_args_complete.extend(proc_args_initial)

        cs.callproc(proc_name, procs_args_complete)

        top_books = [r.fetchall() for r in cs.stored_results()]

        return top_books

In [12]:
top20_books = get_top_recommendation_overall(cs=cs)

In [13]:
for i in top20_books[0]:
    print(i)

(136251, 'Harry Potter and the Deathly Hallows (Harry Potter, #7)', 1784684, 4.62)
(1, 'Harry Potter and the Half-Blood Prince (Harry Potter, #6)', 1713866, 4.54)
(5, 'Harry Potter and the Prisoner of Azkaban (Harry Potter, #3)', 1876252, 4.53)
(6, 'Harry Potter and the Goblet of Fire (Harry Potter, #4)', 1792561, 4.53)
(2, 'Harry Potter and the Order of the Phoenix (Harry Potter, #5)', 1766895, 4.47)
(3, "Harry Potter and the Sorcerer's Stone (Harry Potter, #1)", 4765497, 4.45)
(4667024, 'The Help', 1565302, 4.45)
(13496, 'A Game of Thrones (A Song of Ice and Fire, #1)', 1359501, 4.45)
(15881, 'Harry Potter and the Chamber of Secrets (Harry Potter, #2)', 1821802, 4.38)
(370493, 'The Giving Tree', 720582, 4.37)
(19063, 'The Book Thief', 1193697, 4.36)
(2767052, 'The Hunger Games (The Hunger Games, #1)', 4899965, 4.34)
(34, 'The Fellowship of the Ring (The Lord of the Rings, #1)', 1813229, 4.34)
(128029, 'A Thousand Splendid Suns', 835172, 4.34)
(3777732, 'City of Glass (The Mortal Inst